# 7.3 Logical and conditional expressions

In [1]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


The values of arithmetic expressions can be tested against each other by comparison
operators:
```
=        equal to
<>       not equal to
<        less than
<=       less than or equal to
>        greater than
>=       greater than or equal to
```
The result of a comparison is either "true" or "false". Thus `T > 1` is true if the parameter
`T` has a value greater than 1, and is false otherwise; and
```
sum {i in ORIG} supply[i] = sum {j in DEST} demand[j]
```
is true if and only if total supply equals total demand.

Comparisons are one example of AMPL's logical expressions, which evaluate to true
or false. Set membership tests using in and within, described in [Section 5.4](../05/5_4_set_membership_operations_and_functions.ipynb),
are another example. More complex logical expressions can be built up with logical operators.
The and operator returns true if and only if both its operands are true, while or
returns true if and only if at least one of its operands is true; the unary operator not
returns false for true and true for false. Thus the expression
```
T >= 0 and T <= 10
```
is only true if `T` lies in the interval `[0, 10]`, while the following from [Section 5.5](../05/5_5_indexing_expressions.ipynb),
```
i in MAXREQ or n_min[i] > 0
```
is true if i is a member of `MAXREQ`, or `n_min[i]` is positive, or both. Where several
operators are used together, any comparison, membership or arithmetic operator has
higher precedence than the logical operators; and has higher precedence than or, while
not has higher precedence than either. Thus the expression
```
not i in MAXREQ or n_min[i] > 0 and n_min[i] <= 10
```

is interpreted as
```
(not (i in MAXREQ)) or ((n_min[i] > 0) and (n_min[i] <= 10))
```
Alternatively, the not in operator could be used:
```
i not in MAXREQ or n_min[i] > 0 and n_min[i] <= 10
```
The precedences are summarized in [Table 7-1](../07/7_2_arithmetic_expressions.ipynb#table-7-1), which also gives alternative forms.

Like + and \*, the operators or and and have iterated versions. The iterated or is
denoted by exists, and the iterated and by forall. For example, the expression
```
exists {i in ORIG} demand[i] > 10
```
is true if and only if at least one origin has a demand greater than 10, while
```
forall {i in ORIG} demand[i] > 10
```
is true if and only if every origin has demand greater than 10.

Another use for a logical expression is as an operand to the conditional or if-thenelse
operator, which returns one of two different arithmetic values depending on
whether the logical expression is true or false. Consider the two collections of inventory
balance constraints in the multiperiod production model of [Figure 5-3](../05/5_6_ordered_sets.ipynb#fig-5-3):
```
subject to Balance0 {p in PROD}:
  Make[p,first(WEEKS)] + inv0[p]
     = Sell[p,first(WEEKS)] + Inv[p,first(WEEKS)];
subject to Balance {p in PROD, t in WEEKS: ord(t) > 1}:
  Make[p,t] + Inv[p,prev(t)] = Sell[p,t] + Inv[p,t];
```
The `Balance0` constraints are basically the `Balance` constraints with `t` set to
`first(WEEKS)`. The only difference is in the second term, which represents the previous
week's inventory; it is given as `inv0[p]` for the first week (in the `Balance0` constraints)
but is represented by the variable `Inv[p,prev(t)]` for subsequent weeks (in
the `Balance` constraints). We would like to combine these constraints into one
declaration, by having a term that takes the value `inv0[p]` when `t` is the first week, and takes
the value `Inv[p,prev(t)]` otherwise. Such a term is written in AMPL as:
```
if t = first(WEEKS) then inv0[p] else Inv[p,prev(t)]
```
Placing this expression into the constraint declaration, we can write
```
subject to Balance {p in PROD, t in WEEKS}:
  Make[p,t] +
     (if t = first(WEEKS) then inv0[p] else Inv[p,prev(t)])
	= Sell[p,t] + Inv[p,t];
```
This form communicates the inventory balance constraints more concisely and directly
than two separate declarations.

The general form of a conditional expression is
```
if a then b else c
```

where `a` is a logical expression. If `a` evaluates to true, the conditional expression takes the
value of `b`; if `a` is false, the expression takes the value of `c`. If `c` is zero, the `else c` part
can be dropped. Most often `b` and `c` are arithmetic expressions, but they can also be string
or set expressions, so long as both are expressions of the same kind. Because then and
else have lower precedence than any other operators, a conditional expression needs to
be parenthesized (as in the example above) unless it occurs at the end of a statement.

AMPL also has an if-then-else for use in programming; like the conditional statements
in many programming languages, it executes one or another block of statements
depending on the truth of some logical expression. We describe it with other AMPL programming
features in Chapter 13 xTODO. The if-then-else that we have described here is
not a statement, but rather an expression whose value is conditionally determined. It
therefore belongs inside a declaration, in a place where an expression would normally be
evaluated.

xTODO ==> Mention new chapter about logic or MP
